# I. 데이터 품질 테스트

📌 HDFS 대신 로컬 데이터를 사용하여 품질 테스트를 진행하는 이유

1. 현재 수행하는 작업(중복 검사, 파일 확인 등)은 대규모 분산 처리가 필요 없는 로컬 연산임.
   - HDFS는 대용량 데이터를 분산 저장·처리하는 데 적합하지만, 수천 건 정도면 로컬에서 처리해도 충분함.

2. I/O 속도 향상
   - HDFS에서 데이터를 읽어오는 과정에서 네트워크 지연과 변환 과정(예: Spark의 binaryFile)이 발생함.
   - 우분투 폴더에 있는 파일을 바로 가져오면 I/O 속도를 줄이고 빠르게 처리 가능함.
   - Spark 대신 Pandas 사용 가능

3. 데이터 품질 검사는 Pandas로도 충분히 가능하며, 로컬 데이터에서 Pandas를 쓰면 Spark보다 빠르게 탐색할 수 있음.

## 1. WAV, JSON 데이터 로드 및 중복 검사

In [1]:
import os

data_dir = "/home/ubuntu/data"

json_files = []
wav_files = []

# 하위 폴더까지 포함하여 전체 탐색
for root, _, files in os.walk(data_dir):
    for file in files:
        file_path = os.path.join(root, file)
        if file.endswith(".json"):
            json_files.append(file_path)
        elif file.endswith(".wav"):
            wav_files.append(file_path)

# 파일 개수 출력
print(f"📂 JSON 파일 개수: {len(json_files)}")
print(f"📂 WAV 파일 개수: {len(wav_files)}")

# JSON 파일 일부 출력 (최대 10개)
print("\n📂 JSON 파일 목록 (일부):")
for file in json_files[:5]:  
    print(file)

# WAV 파일 일부 출력 (최대 10개)
print("\n📂 WAV 파일 목록 (일부):")
for file in wav_files[:5]:  
    print(file)

📂 JSON 파일 개수: 18048
📂 WAV 파일 개수: 18048

📂 JSON 파일 목록 (일부):
/home/ubuntu/data/test_data/label_data_test/2.Motorcycle/4.horn_of_motorcycle/2.motorcycle_horn_39554.json
/home/ubuntu/data/test_data/label_data_test/2.Motorcycle/4.horn_of_motorcycle/2.motorcycle_horn_83873.json
/home/ubuntu/data/test_data/label_data_test/2.Motorcycle/4.horn_of_motorcycle/2.motorcycle_horn_83258.json
/home/ubuntu/data/test_data/label_data_test/2.Motorcycle/4.horn_of_motorcycle/2.motorcycle_horn_83833.json
/home/ubuntu/data/test_data/label_data_test/2.Motorcycle/4.horn_of_motorcycle/2.motorcycle_horn_84352.json

📂 WAV 파일 목록 (일부):
/home/ubuntu/data/test_data/raw_data_test/2.Motorcycle/4.horn_of_motorcycle/2.motorcycle_horn_85080_1.wav
/home/ubuntu/data/test_data/raw_data_test/2.Motorcycle/4.horn_of_motorcycle/2.motorcycle_horn_35046_1.wav
/home/ubuntu/data/test_data/raw_data_test/2.Motorcycle/4.horn_of_motorcycle/2.motorcycle_horn_39466_1.wav
/home/ubuntu/data/test_data/raw_data_test/2.Motorcycle/4.horn_of_moto

In [2]:
import os

data_dir = "/home/ubuntu/data"

json_files = []
wav_files = []
json_names = set()
wav_names = set()
duplicate_json = set()
duplicate_wav = set()

# 하위 폴더까지 포함하여 전체 탐색
for root, _, files in os.walk(data_dir):
    for file in files:
        file_path = os.path.join(root, file)
        name, ext = os.path.splitext(file)  # 확장자 제거한 파일명 추출

        if ext == ".json":
            json_files.append(file_path)
            if name in json_names:
                duplicate_json.add(name)  # 중복된 파일명 저장
            else:
                json_names.add(name)

        elif ext == ".wav":
            wav_files.append(file_path)
            if name in wav_names:
                duplicate_wav.add(name)  # 중복된 파일명 저장
            else:
                wav_names.add(name)

# 결과 출력
print(f"📂 JSON 파일 개수: {len(json_files)}")
print(f"⚠️ 중복된 JSON 데이터 개수: {len(duplicate_json)}")

print(f"📂 WAV 파일 개수: {len(wav_files)}")
if duplicate_wav:
    print(f"⚠️ 중복된 WAV 데이터 개수: {len(duplicate_wav)}")
else:
    print("✅ 중복된 WAV 파일 없음") 

📂 JSON 파일 개수: 18048
⚠️ 중복된 JSON 데이터 개수: 0
📂 WAV 파일 개수: 18048
✅ 중복된 WAV 파일 없음
